# Capítulo 6: Árvores de Decisão

## Exercícios

### 1. Qual é a profundidade aproximada de uma Árvore de Decisão treinada (sem restrições) em um conjunto com 1 milhão de instâncias?

A profundidade seria proporcional ao valor do logaritmo do número de instâncias na base 2:
$$P = log_2(N)$$
Dessa forma, a profundidade seria igual a: 
$$log_2(10^6) = 20$$

### 2. O coeficiente Gini de um nó geralmente é menor ou maior do que o dos seus pais? Ele é geralmente menor/maior, ou sempre menor/maior?

É sempre menor

### 3. É uma boa ideia tentar diminuir seu `max_depth` se uma Árvore de Decisão estiver se sobreajustando ao conjunto de treinamento?

Sim, é uma boa ideia

### 4. É uma boa ideia tentar dimensionar as características de entrada se uma Árvore de Decisão estiver se subajustando ao conjunto de treinamento?

### 5. Se treinar uma Árvore de Decisão em um conjunto de treinamento contendo 1 milhão de instâncias demora 1 hora, aproximadamente quanto tempo demorará para treinar outra Árvore de Decisão em um conjunto de treinamento contendo 10 milhões de instâncias?

A ordem de complexidade do treinamento de uma árvore de decisão balanceada é igual a $O(n\times m log(m))$. Com isso pode-se realizar um regra de três para se obter o tempo de treinamento de uma árvore de decisão contendo 10 milhões de instâncias é:
$$n \times 10^6 \times log(10^6) \rightarrow 1 hora $$
$$n \times 10^7 \times log(10^7) \rightarrow X horas $$
Apesar de não ser conhecido o número de características ($n$), como ele é constante para os dois conjuntos de treinamento, acaba não afetando a proporção. Com isso:
$$X = \frac{n \times 10^7 \times log(10^7) \times 1}{n \times 10^6 \times log(10^6)}$$
Logo, demorará cerca de 11 horas e 40 minutos para treinar a outra árvore de decisão. 

### 6. Se o seu conjunto de treinamento contém 100 mil instâncias, a configuração `presort=True` acelerará o treinamento?

Não, pois o ganho de velocidade de treinamento utilizando essa técnica só se faz válido quando o conjunto de treinamento contém um número pequeno de instâncias, cerca de até alguns milhares. 

### 7. Treine e ajuste uma Árvore de Decisão para o conjunto de dados de luas.

  a. Gere um conjunto de dados de luas utilizando `make_moons(n_samples=10000, noise=0.4)`.
  
  b. Com a utilização do `train_test_split()`, divida em um conjunto de treinamento e um conjunto de testes.
  
  c. Utilize a pesquisa de grade com validação cruzada (com a ajuda da classe `GridSearchCV`) para encontrar bons valores de hiperparâmetros para um  `DecisionTreeClassifier`. Dica: tente vários valores para `max_leaf_nodes`.
  
  d. Treine-o no conjunto completo de treinamento utilizando estes hiperparâmetros e meça o desempenho do seu modelo no conjunto de teste. Você deve obter aproximadamente 85% a 87% de acurácia.

In [9]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'max_leaf_nodes': [10, 20, 50, 100, 200, 500],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_tree = grid_search.best_estimator_
print("Melhores hiperparâmetros encontrados:", grid_search.best_params_)

best_tree.fit(X_train, y_train)
y_pred = best_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.2f}")


Melhores hiperparâmetros encontrados: {'max_leaf_nodes': 20, 'min_samples_split': 2}
Acurácia no conjunto de teste: 0.87


### 8. Cultive uma floresta.

 a. Continuando o exercício anterior, gere mil subconjuntos do conjunto de treinamento, cada um contendo 100 instâncias selecionadas aleatoriamente. Dica: você pode utilizar a classe `ShuffleSplit` do Scikit-Learn para isso.

 b. Treine uma Árvore de Decisão em cada subconjunto utilizando os melhores valores do hiperparâmetro encontrados acima. Avalie essas mil Árvores de Decisão no conjunto de testes. Uma vez treinadas em conjuntos menores, essas Árvores de Decisão provavelmente terão um desempenho pior do que a primeira, alcançando apenas 80% de acurácia.
 
 c. Agora vem a mágica. Gere as previsões das mil Árvores de Decisão e mantenha apenas a previsão mais frequente para cada instância do conjunto de testes (você pode utilizar a função `mode()` do SciPy para isso). Isso lhe dá *previsões dos votos majoritários* sobre o conjunto de testes.
 
 d. Avalie estas previsões no conjunto de teste: você deve obter uma acurácia ligeiramente maior que o seu primeiro modelo (cerca de 0,5 a 1,5% a mais). Parabéns, você treinou um classificador de Floresta Aleatória!


**a**. Gerar mil subconjuntos do conjunto de treinamento

In [12]:
from sklearn.model_selection import ShuffleSplit
n_subsets = 1000
subset_size = 100

shuffle_split = ShuffleSplit(n_splits=n_subsets, train_size=subset_size, random_state=42)

mini_sets = []

for mini_train_index, mini_test_index in shuffle_split.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    
    mini_sets.append((X_mini_train, y_mini_train))



**b**. Treinar uma árvore de decisão em cada subconjunto e avaliar no conjunto de testes

In [14]:
import numpy as np
from sklearn.base import clone

forest = [clone(grid_search.best_estimator_) for _ in range(n_subsets)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8012284999999999

**c.** Gerar as previsões das mil árvores de decisão e usar a votação majoritária

In [15]:
Y_pred = np.empty([n_subsets, len(X_test)], dtype = np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [16]:
from scipy.stats import mode

y_pred_majority, n_votes = mode(Y_pred, axis=0)

**d.** Avaliando a acurácia das previsões

In [17]:
accuracy_score(y_test, y_pred_majority.reshape([-1]))

0.872